# Import des librairies 

In [3]:
import pandas as pd 
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import csv
import numpy as np
import re
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from concurrent import futures
import dateparser

import warnings
warnings.filterwarnings('ignore')

data_path="Data/"#Chemin du dossier pour enregistrer le fichier csv

In [4]:
def check_nan(df):
    for i in df.columns.tolist():
        print("Valeurs nan dans "+str(i)+" : "+str(df[i].isna().sum()))
        
def check_unique(df):
    for i in df.columns.tolist():
        print("Valeurs uniques dans "+str(i)+" : "+str(df[i].nunique()))

# Chargement des données déjà existantes

In [5]:
df_ids = pd.read_csv(data_path+"allocine_titre_id.csv")
df_ids.head()

,titre,id
0,avatar : la voie de l'eau,178014
1,les banshees d'inisherin,281293
2,tempête,289305
3,m3gan,266320
4,le tourbillon de la vie,288544


# Scrapping des données de la page Home pour chaque film 

## Fonctions : 

In [4]:
def first_connexion(headless=False) :
    options = webdriver.ChromeOptions()
    
    if headless==True : 
        options.add_argument('-headless')
        options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
    driver.get("https://www.allocine.fr/")
    time.sleep(2)
    driver.get_screenshot_as_file("screenshot.png")
    if headless==False : 
        cookies_box = driver.find_element(By.XPATH, '//*[@id="cmp-main"]/button[2]')
        cookies_box.click()
    time.sleep(5)
    return driver

In [5]:
def get_data_home(movie_id, driver):
    url=f"https://www.allocine.fr/film/fichefilm_gen_cfilm={movie_id}.html"
    data={}
    driver.get(url)
    time.sleep(0.3)
    data["id"] =movie_id
    data["titre"]= driver.find_element_by_xpath('//*[@id="content-layout"]/div[2]/div[1]').text
    
    try : 
        text_data = driver.find_element_by_class_name('meta-body-item.meta-body-info')
    except : 
        print("No text data found")
        print(data["titre"].lower())
        
    if "Date de sortie inconnue".lower() not in text_data.text.lower() : 
        try :     
            data["date_sortie"] = text_data.find_elements_by_class_name("blue-link")[0].text
        except : 
            try : 
                data["date_sortie"] = text_data.find_elements_by_class_name("date")[0].text
            except : 
                pass 
        
    #Support de sortie
    support = text_data.find_elements(By.TAG_NAME,"strong")
    if len(support) != 0 : 
        data["support"] = support[0].text
    
    #Durée
    match = re.search(r"(\d{1,2}h\s\d{1,2}min)", text_data.text)
    if match : 
        data["duree"] =match.group(0)

    #Genres du films
    genres=[]
    for a in text_data.find_elements(By.TAG_NAME,"a") :
        if "genre" in a.get_attribute("href") : 
            genres.append(a.text)
    data["genres"] = genres

    try :
        data["synopsis"] = data["synopsis"] =driver.find_elements(By.ID, "synopsis-details")[0].find_elements(By.CLASS_NAME, "content-txt")[0].text
    except : 
        print(f"{data['titre']} pas de synopsis")
        
    try : 
        divs_eval = driver.find_elements_by_class_name('rating-item-content')
        if len(divs_eval)>0 :
            for elem in divs_eval : 
                if "presse" in elem.text.lower() : 
                    data["note_moyenne_presse"]= elem.find_elements_by_class_name("stareval-note")[0].text
                if "spectateur" in elem.text.lower() :
                    data["note_moyenne_spectateurs"]= elem.find_elements_by_class_name("stareval-note")[0].text
    except Exception as exc: 
        print(data["titre"])
        print(exc)
        
            
    return data 

In [6]:
def init_movies_by_driver(nb_drivers, len_data) :
    drivers_ids_range ={}
    nb_films_by_driver = np.ceil(len_data/nb_drivers)
    for i in range(nb_drivers) :
        drivers_ids_range[i]=  {"start" : int(i * nb_films_by_driver), "end" : int((i+1)*nb_films_by_driver)}
    return drivers_ids_range 

In [7]:
def workload_driver(info_driver) :
    data_processed = df_ids.iloc[info_driver["index_start"]:info_driver["index_end"]].iloc[:100]
    driver= first_connexion(headless=True)#Première connexion
    #Scrapping des données
    new_data = [] 
    for index, id in enumerate(data_processed["id"]) : 
        new_data.append(get_data_home(id, driver))
        if index % 20 == 0 : 
            if info_driver["id"] ==0 : 
                clear_output()
            print(f"Driver n°{info_driver['id']} : {index} films")
                
    driver.close()
    #Sauvegarde des données

    df= pd.DataFrame(new_data)
    temp = pd.read_csv(data_path+f"allocine_home_{info_driver['id']}.csv")
    pd.concat([temp, df]).to_csv(data_path+f"allocine_home_{info_driver['id']}.csv",index=False)

In [8]:
def init_save_files(number) :
    for i in range(number) :
        df_init= pd.DataFrame(columns= ['id', 'titre', 'date_sortie', 'support', 'duree', 'genres', 'synopsis','note_moyenne_presse', 'note_moyenne_spectateurs'])
        df_init.to_csv(data_path+f"allocine_home_{i}.csv",index=False)

### Multiprocessing

In [75]:
driver= first_connexion()
driver.get(f"https://www.allocine.fr/film/fichefilm_gen_cfilm={movie_id}.html")

time.sleep(0.5)

data={}

text_data = driver.find_element_by_class_name('meta-body-item.meta-body-info')


print(text_data.get_attribute("innerHTML"))
print()
print(text_data.text)
print()

if "Date de sortie inconnue".lower() not in text_data.text.lower() : 
    try :     
        data["date_sortie"] = text_data.find_elements_by_class_name("blue-link")[0].text
    except : 
        try : 
            data["date_sortie"] = text_data.find_elements_by_class_name("date")[0].text
        except : 
            pass 

#Support de sortie
support = text_data.find_elements(By.TAG_NAME,"strong")
if len(support) != 0 : 
    data["support"] = support[0].text
    

#Durée du film 
match = re.search(r"(\d{1,2}h\s\d{1,2}min)", text_data.text)
if match : 
    data["duree"] =match.group(0)

#Genres du films
genres=[]
for a in text_data.find_elements(By.TAG_NAME,"a") :
    if "genre" in a.get_attribute("href") : 
        genres.append(a.text)

data["genres"] = genres

data["synopsis"] =driver.find_elements(By.ID, "synopsis-details")[0].find_elements(By.CLASS_NAME, "content-txt")[0].text

print(data)
driver.close()


<span class="date">17 novembre 2022</span>
<strong>
en DVD
</strong>
<span class="spacer">/</span>
<a class="xXx" href="/films/genre-13019/">Western</a>


17 novembre 2022 en DVD / Western

{'date_sortie': '17 novembre 2022', 'support': 'en DVD', 'genres': ['Western'], 'synopsis': 'Une famille de pionniers se bat contre une bande de hors-la-loi qui les terrorise dans leur ferme nouvellement construite dans les plaines du Montana.'}


In [13]:
movie_id =297254

driver= first_connexion()
driver.get(f"https://www.allocine.fr/film/fichefilm_gen_cfilm={movie_id}.html")

time.sleep(0.5)

try : 
    text_data = driver.find_element_by_xpath('//*[@id="content-layout"]/section/div/div[2]/div[1]/div/div[1]').text.split("/")
except : 
    try : 
        text_data= driver.find_element_by_xpath('//*[@id="content-layout"]/section/div/div[3]/div[1]/div/div[1]').text.split("/")
    except : 
        print("Erreur text_data")
print(text_data)

['17 novembre 2022 en DVD ', ' Western']


# Algorithme final

In [18]:
%%time
nb_drivers = 6
nb_films_by_iter =100
nb_movies= len(df_ids)

ids_movies_by_drivers = init_movies_by_driver(nb_drivers, nb_movies)

nb_iterations =(nb_movies//(nb_drivers*nb_films_by_iter))+1

init_save_files(nb_drivers)

#Nombre de fois que les drivers vont se lancer et se fermer
for j in range(nb_iterations) : 
    print("Itération :",j)
    with futures.ThreadPoolExecutor() as executor: 
        future_results = [ executor.submit(workload_driver,{'id':i, 'index_start' :ids_movies_by_drivers[i]["start"]+nb_films_by_iter*j , 'index_end':ids_movies_by_drivers[i]["end"]})  for i in range(nb_drivers) ] 
        for future_result in future_results: 
            try:        
                future_result = future_result.result() # can use `timeout` to wait max seconds for each thread               
                #... do something with the test_result
            except Exception as exc: # can give a exception in some thread, but 
                print("thread generated an exception",exc)
                break;

#Concaténation et enregirstrement des données
df=pd.DataFrame()
for i in range(nb_drivers): 
    df=pd.concat([df, pd.read_csv(data_path+f"allocine_home_{i}.csv")])
df.to_csv(data_path+"allocine_home.csv",index=False)

Driver n°0 : 20 films
LE PETIT POUCET pas de synopsis
VEERA SIMHA REDDY pas de synopsis
STAYING ALIVE pas de synopsis
Driver n°5 : 20 films
Driver n°2 : 20 films
AUX PORTES DE L'AU-DELÀ pas de synopsis
Driver n°4 : 20 films
SURVIVAL GAME pas de synopsis
CPU times: total: 15min 27s
Wall time: 2h 27min 14s


## Lecture des données

In [9]:
df=pd.read_csv(data_path+"allocine_home.csv")
print("Shape df :",df.shape)
print()
check_nan(df)
print()
check_unique(df)
df.head()

Shape df : (19882, 9)

Valeurs nan dans id : 0
Valeurs nan dans titre : 0
Valeurs nan dans date_sortie : 0
Valeurs nan dans support : 69
Valeurs nan dans duree : 202
Valeurs nan dans genres : 0
Valeurs nan dans synopsis : 280
Valeurs nan dans note_moyenne_presse : 10349
Valeurs nan dans note_moyenne_spectateurs : 113

Valeurs uniques dans id : 19882
Valeurs uniques dans titre : 19051
Valeurs uniques dans date_sortie : 6028
Valeurs uniques dans support : 28
Valeurs uniques dans duree : 213
Valeurs uniques dans genres : 1798
Valeurs uniques dans synopsis : 19593
Valeurs uniques dans note_moyenne_presse : 41
Valeurs uniques dans note_moyenne_spectateurs : 39


,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs
0,178014,AVATAR : LA VOIE DE L'EAU,14 décembre 2022,en salle,3h 12min,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3"
1,281293,LES BANSHEES D'INISHERIN,28 décembre 2022,en salle,1h 54min,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9"
2,289305,TEMPÊTE,21 décembre 2022,en salle,1h 49min,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9"
3,266320,M3GAN,28 décembre 2022,en salle,1h 42min,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9"
4,288544,LE TOURBILLON DE LA VIE,21 décembre 2022,en salle,2h 01min,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9"


## Nettoyage 

## Mise en minuscule des titres de films

In [10]:
df["titre"] = df["titre"].str.lower()

## Parsing des dates 

In [13]:
df['date_sortie'] = df['date_sortie'].apply(dateparser.parse)

## Conversion de la durée en minutes 

In [28]:
df["duree"]=df['duree'].apply(lambda x: int(x.split('h')[0]) * 60 + int(x.split('h')[1].split('min')[0]) if (np.all(pd.notnull(x))) else x)

In [32]:
df.head()

,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3"
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9"
2,289305,tempête,2022-12-21,en salle,109.0,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9"
3,266320,m3gan,2022-12-28,en salle,102.0,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9"
4,288544,le tourbillon de la vie,2022-12-21,en salle,121.0,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9"


## Sauvegarde 

In [31]:
df.to_csv(data_path+"allocine_home_cleaned.csv",index=False)